In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By

import requests
from bs4 import BeautifulSoup

In [4]:
def get_article_links(search_term, limit = 100, driver = webdriver.Firefox()):
    
    url_path = f'https://www.investing.com/search/?q={search_term}&tab=news'
    driver.get(url_path)

    links = []

    pos = 0
    while len(driver.find_elements(by=By.CLASS_NAME, value="articleItem")) < limit:
        pos += 500
        driver.execute_script(f'window.scrollTo(0, {pos})')

    article_items = driver.find_elements(by=By.CLASS_NAME, value="articleItem")  
    for article in article_items:
        if article.get_attribute("class") != "js-article-item articleItem     ": 
            link = article.find_element(by=By.CLASS_NAME, value="title")         
            links.append(link.get_attribute("href"))             
            
    driver.quit()
    return links

article_links = get_article_links("msft")

In [7]:
test_list = article_links[:]
test_list

def get_article_text(link):

    response = requests.get(link)

    if response.status_code == 200:

        soup = BeautifulSoup(response.text, 'html.parser')
        article = soup.find('div', class_='WYSIWYG articlePage')

        script = article.find("script")
        if script != None:
            script.extract()
        img_carousel = article.find('div', class_='imgCarousel')
        if img_carousel != None:
            img_carousel.extract()
        related_instruments_wrapper = article.find('div', class_='relatedInstrumentsWrapper')
        if related_instruments_wrapper != None:
            related_instruments_wrapper.extract()
        paragraph = article.find("p")
        if paragraph != None:
            em = paragraph.find("em")
            if em != None:
                paragraph.extract()

        text_inside_div = article.get_text()
        return text_inside_div.strip()
    
    else:
        return None

def get_article_texts(links):
    return [get_article_text(link) for link in links]

get_article_texts(test_list)

["BEIJING (Reuters) - Shanghai authorities told Microsoft (NASDAQ:MSFT)'s visiting president on Tuesday they want his company to promote artificial intelligence technology to boost businesses there, the Chinese financial hub's government said. \nChen Jining, Shanghai's Communist party Secretary, made the remarks while meeting Microsoft President and Vice Chair Brad Smith, the government said. Shanghai was also open to Microsoft collaborating on studying technology-related governance frameworks and standards, Chen said.",
 'By Foo Yun Chee, Supantha Mukherjee and Martin Coulter BRUSSELS/STOCKHOLM/LONDON (Reuters) - European Union ambitions to take a lead in landmark rules for artificial intelligence hang in the balance as member states and lawmakers meet on Wednesday to try to hammer out a deal on biometric surveillance and how to regulate systems like ChatGPT. If agreed, the EU\'s first-of-a-kind AI Act, which was proposed by the European Commission two years ago, could serve as the be